In [1]:
import numpy as np
import pandas as pd
import datetime
import pyreadr
import pandas as pd
from IPython.display import display

from sklearn.preprocessing import OneHotEncoder

from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Unified COVID-19 Dataset

In [2]:
policies_df = pyreadr.read_r('Policy.RData') # also works for Rds
print(policies_df.keys()) # let’s check what objects we got

odict_keys(['Policy'])


In [3]:
policies_df =policies_df['Policy']

In [4]:
policies_df.head()

,ID,Date,PolicyType,PolicyValue,PolicyFlag,PolicyNotes,PolicySource
0,AD,2020-01-01,C1,0.0,NaN,NaN,OxCGRT
1,AD,2020-01-01,C2,0.0,NaN,NaN,OxCGRT
2,AD,2020-01-01,C3,0.0,NaN,NaN,OxCGRT
3,AD,2020-01-01,C4,0.0,NaN,NaN,OxCGRT
4,AD,2020-01-01,C5,0.0,NaN,NaN,OxCGRT


# Coronanet dataset

In [5]:
# import data
df = pd.read_csv('coronanet_release_allvars.csv', encoding='mac_roman')

In [6]:
# colonnes
df.columns

Index(['Unnamed: 0', 'record_id', 'policy_id', 'entry_type', 'correct_type',
       'update_type', 'update_level', 'description', 'date_announced',
       'date_start', 'date_end', 'country', 'ISO_A3', 'ISO_A2',
       'init_country_level', 'domestic_policy', 'province', 'city', 'type',
       'type_sub_cat', 'type_text', 'institution_status', 'target_country',
       'target_geog_level', 'target_region', 'target_province', 'target_city',
       'target_other', 'target_who_what', 'target_direction',
       'travel_mechanism', 'compliance', 'enforcer', 'index_high_est',
       'index_med_est', 'index_low_est', 'index_country_rank', 'link',
       'date_updated', 'recorded_date', 'confirmed_cases', 'deaths',
       'recovered', 'new_tests', 'total_tests', 'total_tests_per_thousand',
       'tests_units', 'Rank_FP', 'Score_FP', 'state_IDC', 'muni_IDC',
       'dispersive_IDC', 'constraining_IDC', 'inclusive_IDC', 'sfi_SFI',
       'ti_cpi_TI', 'pop_WDI_PW', 'gdp_WDI_PW', 'gdppc_WDI_PW',
 

In [7]:
# NaNs
df.isnull().sum()

Unnamed: 0                      0
record_id                       0
policy_id                       0
entry_type                      0
correct_type                    0
update_type                 34546
update_level                36539
description                     0
date_announced                 14
date_start                      0
date_end                    23998
country                         0
ISO_A3                        243
ISO_A2                        267
init_country_level            162
domestic_policy                 0
province                    20635
city                        43546
type                            0
type_sub_cat                13819
type_text                   43425
institution_status          32395
target_country               2159
target_geog_level           35938
target_region               45277
target_province             21264
target_city                 43083
target_other                45640
target_who_what             11137
target_directi

In [8]:
df['compliance'].nunique()

31

In [9]:
df_2 = df[['policy_id','entry_type','correct_type', 
           'description', 'date_start','date_end','country', 
           'type', 'ISO_A3','init_country_level', 'domestic_policy',
           'type_sub_cat', 'compliance', 'enforcer']]

In [10]:
df_2.shape@

SyntaxError: invalid syntax (<ipython-input-10-ab36016514c1>, line 1)

In [ ]:
df_2.isnull().sum()

In [ ]:
df_2[df_2['compliance'] == 'Mandatory with Fines,Voluntary/Recommended but No Penalties']

## get_dummies sur la colonne type

In [ ]:
# commençons par le type (mesures plus simples pour les restrictions pour commencer et possibilité d'encodage)
print(df_2['type'].unique())
print(df_2['type'].nunique())

In [ ]:
df_2 = pd.get_dummies(df_2, columns=["type"]).head()

## Encodage des données compliance

In [ ]:
for _ in df['compliance'].unique():
    print('-'*50)
    print(_)

In [ ]:
mask = df_2['compliance'].value_counts() > 1000

In [ ]:
df_3 = df_2[df_2['compliance'].isin(mask.index)]

In [ ]:
def handle_compliance(x):
    if 'Voluntary/Recommended but No Penalties' in x:
        return 0
    if 'Mandatory with Fines':
        return 1
    if 'Mandatory with Legal Penalties' in x:
        return 2
    if 'Mandatory with Exceptions' in x:
        return 3

In [ ]:
df_3['n_compliance'] = df_3['compliance'].map(handle_compliance)

In [ ]:
df_3['n_compliance'].nunique()

 ## Vectorization des données description (pour plus tard)

In [ ]:
def clean(x):
    x = x.lower()
    x = ''.join([letter for letter in x if not letter.isdigit()])
    for p in string.punctuation:
        x = x.replace(p, " ")
    stops = set(stopwords.words("english"))
    tokens = word_tokenize(x)
    tokens = ' '.join([token for token in tokens if token not in stops])
    return tokens

In [ ]:
# données description
df_2['clean_description'] = df_2.description.apply(clean)
df_2['clean_description'] = df_2['clean_description'].astype('str')